# 수원서베이 2024 데이터 정리 & 코드북 라벨 매핑 노트북

이 노트북은 다음을 수행합니다.

1. **엑셀 데이터의 1행(변수 코드)이 본문에 들어간 문제**를 수정하여 컬럼명을 변수 코드로 재설정
2. 코드북(`Variable Name`/`Value Code`/`Value Label`)을 사용하여 **값 라벨 매핑**
3. **라벨링된 데이터**를 CSV/XLSX로 내보내기
4. **매핑 적용 커버리지 리포트** 출력

> 경로는 이 노트북이 위치한 환경 기준 `/mnt/data/`를 사용합니다.


## 0. 경로/패키지 설정 및 원본 미리보기

In [10]:

from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().parent
DATA_PATH = ROOT / "data" / "internal" / "1. 수원서베이" / "(HRC250604) 2024년 수원서베이 용역_공개용 데이터" / "(HRC250604) 2024년 수원서베이 용역_공개용 데이터(엑셀).xlsx"
CODEBOOK_PATH = ROOT / "data" / "internal" / "1. 수원서베이" / "(HRC250604) 2024년 수원서베이 용역_공개용 데이터" / "(HRC250604) 2024년 수원서베이 용역_공개용 데이터_코드북.xlsx"

OUT_DIR = ROOT / "output" / "1. 수원서베이"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 원본 로드 (첫 시트)
data_raw = pd.read_excel(DATA_PATH, sheet_name=0)
codebook = pd.read_excel(CODEBOOK_PATH, sheet_name=0)

print("Raw data shape:", data_raw.shape)
print("Codebook shape:", codebook.shape)

# 데이터 앞부분 확인
display(data_raw.head(3))
display(codebook.head(10))


Raw data shape: (3058, 481)
Codebook shape: (2789, 4)


,응답자 PID,(가구용) 문0. 가구 정보 - 1. 거주 주택 유형,(가구용) 문0. 가구 정보 - 2. 1인 가구 여부,(가구용) 문0. 가구 정보 - 3. 맞벌이 여부(가구주 기준),(가구용) 문0. 응답자 성별,(가구용) 문0. 응답자 출생연도,(가구용) 문0. 총 가구원 수,(가구용) 문0. 가구원1 가구대표자(응답자)와의 관계,(가구용) 문0. 가구원1 가구대표자(응답자)와의 관계_기타,(가구용) 문0. 가구원1 성별,...,정책 관심도,수원시정 만족도,한 주간 삶의 질_평균(7점),영역별 행복 정도_평균(10점),환경 만족도_평균(7점),한 주간 삶의 질_평균(100점),영역별 행복 정도_평균(100점),환경 만족도_평균(100점),모수가중치,표본가중치
0,PID,H0a1,H0a2,H0a3,gender,birth,H0a4,H0a5n1,H0a5opn1,H0a9n1,...,SCORE1,SCORE2,MQ1,MQ2,MQ4,MHQ1,MHQ2,MHQ4,wg,ws
1,29715,1,1,3,1,1992,1,1,NaN,1,...,2,1,4,5.6,4.6,50,56,60.0,241,0.729643
2,51295,1,2,1,2,1981,4,2,NaN,1,...,2,3,4.857143,8,5.3,64.285714,80,71.666667,202.384615,0.612732


,Variable Name,Variable Label,Value Code,Value Label
0,PID,응답자 PID,NaN,NaN
1,H0a1,(가구용) 문0. 가구 정보 - 1. 거주 주택 유형,1.0,아파트
2,H0a1,(가구용) 문0. 가구 정보 - 1. 거주 주택 유형,2.0,그 외
3,H0a2,(가구용) 문0. 가구 정보 - 2. 1인 가구 여부,1.0,1인 가구
4,H0a2,(가구용) 문0. 가구 정보 - 2. 1인 가구 여부,2.0,다인 가구
5,H0a3,(가구용) 문0. 가구 정보 - 3. 맞벌이 여부(가구주 기준),1.0,맞벌이
6,H0a3,(가구용) 문0. 가구 정보 - 3. 맞벌이 여부(가구주 기준),2.0,외벌이
7,H0a3,(가구용) 문0. 가구 정보 - 3. 맞벌이 여부(가구주 기준),3.0,해당 없음
8,gender,(가구용) 문0. 응답자 성별,NaN,NaN
9,birth,(가구용) 문0. 응답자 출생연도,NaN,NaN


## 1. 헤더/첫 행 문제 수정 (변수 코드로 컬럼 교체)

In [11]:

# 상황: 현재 data_raw의 컬럼은 '질문 라벨'이며, 본문 첫 행(인덱스 0)에 '변수 코드'가 들어가 있음.
# 조치: 0번째 행을 컬럼명으로 사용하고, 그 행은 제거.

# 1) 첫 행을 변수명으로 추출
first_row_as_varnames = list(data_raw.iloc[0, :])

# 2) 컬럼명을 변수 코드로 교체
data_fixed = data_raw.copy()
data_fixed.columns = first_row_as_varnames

# 3) 변수 코드가 들어간 첫 행 제거
data_fixed = data_fixed.iloc[1:, :].reset_index(drop=True)

print("Fixed data shape:", data_fixed.shape)
display(data_fixed.head(3))


Fixed data shape: (3057, 481)


,PID,H0a1,H0a2,H0a3,gender,birth,H0a4,H0a5n1,H0a5opn1,H0a9n1,...,SCORE1,SCORE2,MQ1,MQ2,MQ4,MHQ1,MHQ2,MHQ4,wg,ws
0,29715,1,1,3,1,1992,1,1,NaN,1,...,2,1,4,5.6,4.6,50,56,60.0,241,0.729643
1,51295,1,2,1,2,1981,4,2,NaN,1,...,2,3,4.857143,8,5.3,64.285714,80,71.666667,202.384615,0.612732
2,52982,1,2,2,1,1977,5,1,NaN,1,...,2,3,4.571429,5.6,5.1,59.52381,56,68.333333,212,0.641843


## 2. 코드북 매핑 딕셔너리 생성 (문자열/숫자형 키 이중 매핑)

In [12]:

# 코드북 표준화: 컬럼명 통일
codebook = codebook.rename(columns={
    "Variable Name": "variable",
    "Variable Label": "label",
    "Value Code": "code",
    "Value Label": "value_label"
})

# 매핑 dict 생성: 변수별 {code -> value_label}
# 타입 이슈 방지 위해 str 변환 매핑 + 원래 타입 매핑 둘 다 구성
code_mappings_str = {}
code_mappings_num = {}

for var, g in codebook.groupby("variable"):
    # code가 NaN인 행은 값 매핑이 아님(변수 설명행) -> 제외
    g2 = g.dropna(subset=["code", "value_label"])
    if g2.empty:
        continue
    # 문자열 기반 매핑
    code_mappings_str[var] = dict(zip(g2["code"].astype(str), g2["value_label"].astype(str)))
    # 숫자형 기반 매핑 (가능한 경우만)
    try:
        g2_num = g2[g2["code"].apply(lambda x: pd.api.types.is_number(x))]
        if not g2_num.empty:
            code_mappings_num[var] = dict(zip(g2_num["code"].astype(float), g2_num["value_label"].astype(str)))
    except Exception:
        pass

len(code_mappings_str), len(code_mappings_num)


(387, 387)

## 3. 전 컬럼에 라벨 매핑 적용

In [13]:

data_labeled = data_fixed.copy()
covered_cols = []
uncovered_cols = []

for col in data_labeled.columns:
    if col in code_mappings_str or col in code_mappings_num:
        s = data_labeled[col]
        s_mapped = s.copy()

        # 1) 문자열 매핑 시도
        try:
            s_mapped_str = s.astype(str).map(code_mappings_str.get(col, {}))
        except Exception:
            s_mapped_str = pd.Series([np.nan]*len(s), index=s.index)

        # 2) 숫자형 매핑 시도
        try:
            s_numeric = pd.to_numeric(s, errors="coerce")
            s_mapped_num = s_numeric.map(code_mappings_num.get(col, {}))
        except Exception:
            s_mapped_num = pd.Series([np.nan]*len(s), index=s.index)

        # 우선순위: 숫자형 매핑 -> 문자열 매핑 -> 원본
        s_final = s_mapped_num.where(~s_mapped_num.isna(), s_mapped_str)
        s_final = s_final.where(~s_final.isna(), s)

        data_labeled[col] = s_final
        covered_cols.append(col)
    else:
        uncovered_cols.append(col)

print(f"매핑 적용 컬럼 수: {len(covered_cols)} / 전체 {data_labeled.shape[1]}")
print("예시(앞 3행):")
display(data_labeled.head(3))


매핑 적용 컬럼 수: 387 / 전체 481
예시(앞 3행):


,PID,H0a1,H0a2,H0a3,gender,birth,H0a4,H0a5n1,H0a5opn1,H0a9n1,...,SCORE1,SCORE2,MQ1,MQ2,MQ4,MHQ1,MHQ2,MHQ4,wg,ws
0,29715,아파트,1인 가구,해당 없음,1,1992,1,본인,NaN,남,...,보통,불만족,4,5.6,4.6,50,56,60.0,241,0.729643
1,51295,아파트,다인 가구,맞벌이,2,1981,4,본인의 배우자,NaN,남,...,보통,만족,4.857143,8,5.3,64.285714,80,71.666667,202.384615,0.612732
2,52982,아파트,다인 가구,외벌이,1,1977,5,본인,NaN,남,...,보통,만족,4.571429,5.6,5.1,59.52381,56,68.333333,212,0.641843


## 4. 매핑 커버리지 요약/점검

In [14]:

# 매핑 커버리지/표본 분포 리포트
report_rows = []
for col in data_labeled.columns:
    # 범주 수/결측률/고유값 수 파악
    s = data_labeled[col]
    nunique = s.nunique(dropna=True)
    na_rate = s.isna().mean()
    # 코드북 매핑 존재 여부
    has_mapping = col in code_mappings_str or col in code_mappings_num
    report_rows.append({
        "variable": col,
        "has_mapping": has_mapping,
        "nunique": int(nunique),
        "na_rate": round(float(na_rate), 4)
    })

coverage_df = pd.DataFrame(report_rows).sort_values(["has_mapping", "nunique"], ascending=[False, False])
display(coverage_df.head(20))

# 요약
summary = {
    "total_columns": int(data_labeled.shape[1]),
    "mapped_columns": int(coverage_df["has_mapping"].sum()),
    "unmapped_columns": int((~coverage_df["has_mapping"]).sum())
}
summary


,variable,has_mapping,nunique,na_rate
449,DM32,True,44,0.0000
270,Q25m1,True,18,0.0000
271,Q25m2,True,16,0.1165
272,Q25m3,True,15,0.2879
273,Q25m4,True,14,0.4550
117,H10r1,True,13,0.0000
118,H10r2,True,13,0.0000
119,H10r3,True,13,0.0000
274,Q25m5,True,13,0.6307
112,H8,True,12,0.0000


{'total_columns': 481, 'mapped_columns': 387, 'unmapped_columns': 94}

## 5. CSV/XLSX 내보내기

In [15]:

# 결과 저장
csv_path = OUT_DIR / "suwon_2024_labeled.csv"
xlsx_path = OUT_DIR / "suwon_2024_labeled.xlsx"

# 엑셀은 시트 구조로 메타+리포트 포함
with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as w:
    data_labeled.to_excel(w, sheet_name="data_labeled", index=False)
    coverage_df.to_excel(w, sheet_name="coverage_report", index=False)
    pd.DataFrame([summary]).to_excel(w, sheet_name="summary", index=False)

data_labeled.to_csv(csv_path, index=False, encoding="utf-8-sig")

print("Saved:", csv_path)
print("Saved:", xlsx_path)


Saved: d:\workspace\dacon_sri\output\1. 수원서베이\suwon_2024_labeled.csv
Saved: d:\workspace\dacon_sri\output\1. 수원서베이\suwon_2024_labeled.xlsx
